# Cvičení: Lineární regrese
Zdroj: [Jiří Pešík. *Python pro data science.*](https://github.com/pesikj/PythonProDataScience)
(upraveno)

## Pojišťovna

V souboru `expenses.csv` najdeš informace o platbách za pojištěnce jedné pojišťovny: věk, pohlaví, BMI (index počítaný jako hmotnost dělená výškou), počet dětí, kuřák/nekuřák, region a platby za pojištěnce. ...

In [1]:
import pandas as pd
import statsmodels.formula.api as smf

In [2]:
data = pd.read_csv("expenses.csv")
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


## Základní model

Vytvoř regresní model, který odhadne platby za pojištěnce na základě jeho věku a indexu BMI.

In [3]:
formula = "charges ~ age + bmi"
model = smf.ols(formula=formula, data=data)

res = model.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     88.60
Date:                Sun, 17 Nov 2024   Prob (F-statistic):           7.39e-37
Time:                        14:00:45   Log-Likelihood:                -14394.
No. Observations:                1338   AIC:                         2.879e+04
Df Residuals:                    1335   BIC:                         2.881e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -6424.8046   1744.091     -3.684      0.000   -9846.262   -3003.347
age          241.9308     22.298     10.850      0.000     198.187     285.674
bmi          332.9651     51.374      6.481      0.000     232.182     433.748
==============================================================================
Omnibus:                      321.874   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              592.574
Skew:                           1.511   Prob(JB):                    2.11e-129
Kurtosis:                       4.223   Cond. No.                         287.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Vylepšený model

Model zatím není příliš přesný, což lze posoudit pomocí hodnoty R-squared (koeficient determinace), která by ideálně měla být co nejblíže hodnotě 1, ale zatím dosahuje pouze nízké hodnoty: **R-squared: 0.117**

Zkusme do modelu přidat další proměnnou, a to informaci o tom, jestli je pojištěnec kuřák, nebo nekuřák (sloupec `smoker`). K tomu nejdřív potřebujeme převést tuto kategoriální proměnnou na číselnou proměnnou, tzv. dummy proměnnou:

### Vytvoření dummy proměnných

V regresi se často využívá metoda označovaná jako One Hot Encoding, která slouží ke zpracování nečíselných (kategoriálních) dat. Metodu aplikujeme tak, že vytvoříme sloupec pro každou hodnotu, které kategoriálních hodnota může nabýt. Pro každý řádek je pak hodnota příslušného sloupce 1 a ostatních sloupců 0. Převeď kategoriální proměnnou "smoker" na dummy proměnné pomocí příkazu `data = pd.get_dummies(data, columns=["smoker"])`.

In [4]:
data = pd.get_dummies(data, columns=["smoker"])
# ...parametr columns určuje, které sloupce převést na dummy proměnné

data.head()

,age,sex,bmi,children,region,charges,smoker_no,smoker_yes
0,19,female,27.900,0,southwest,16884.92400,False,True
1,18,male,33.770,1,southeast,1725.55230,True,False
2,28,male,33.000,3,southeast,4449.46200,True,False
3,33,male,22.705,0,northwest,21984.47061,True,False
4,32,male,28.880,0,northwest,3866.85520,True,False


### Přidání další proměnné do modelu

Následně sloupec `smoker_yes` přidej jako další nezávislou proměnnou do regresního modelu.

In [5]:
formula = "charges ~ age + bmi + smoker_yes"

mod = smf.ols(formula=formula, data=data)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.747
Model:                            OLS   Adj. R-squared:                  0.747
Method:                 Least Squares   F-statistic:                     1316.
Date:                Sun, 17 Nov 2024   Prob (F-statistic):               0.00
Time:                        14:00:45   Log-Likelihood:                -13557.
No. Observations:                1338   AIC:                         2.712e+04
Df Residuals:                    1334   BIC:                         2.714e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept          -1.168e+04    937.569    -12.454      0.000   -1.35e+04   -9837.561
smoker_yes[T.True]  2.382e+04    412.867     57.703      0.000     2.3e+04    2.46e+04
age                  259.5475     11.934     21.748      0.000     236.136     282.959
bmi                  322.6151     27.487     11.737      0.000     268.692     376.538
==============================================================================
Omnibus:                      299.709   Durbin-Watson:                   2.077
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              710.137
Skew:                           1.213   Prob(JB):                    6.25e-155
Kurtosis:                       5.618   Cond. No.                         289.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Výsledky

Přidáním informace o kuřáctví se výrazně zvýšil koeficient determinace: **R-squared 0.117 -> 0.747**